In [ ]:
%matplotlib inline
import xarray as xr
from matplotlib import cycler
from xnoah import swap_coord
from datetime import datetime


plt.style.use(["noah", "seaborn-darkgrid"])

I was having a problem with the SCAM, IOP, and NN data not being aligned in the 2.0-paper-plots notebook. I suspect that this is because the time coordinate is not aligned between the different datasets. For the CAM runs, I chose the 1999-01-01 as the arbitrary start date of the simulation as you can see here. First let's load the data:

In [ ]:
nn = xr.open_dataset("../data/output/columns.nc")
scam = xr.open_dataset("../data/output/scam.nc")
iop = xr.open_dataset("../data/processed/iop.nc")

This is the start date of the simulation

In [ ]:
iop.bdate

Let's make sure these data are defined on the same horizontal coordinates, 'x',  and 'y'

In [ ]:
iop = swap_coord(iop, {'lon': 'x', 'lat': 'y'})
nn = nn.sel(y=scam.y).assign_coords()
iop = iop.sel(y=scam.y)

Need to make the time coordinates of iop, scam and nn match. I am just going to convert the nn and iop to have date formats.

In [ ]:
bdate = np.datetime64('1999-01-01')
time = (nn.time.values  * 86400).astype('timedelta64[s]') + bdate

nn = nn.assign_coords(time=time)
iop = iop.assign_coords(tsec=time).rename({'tsec': 'time'})

Now that all the data are aligned in time, we should convert the time coordinates back to days from simulation start, because the dates have no significance. We only used the dates to help align the data.

In [ ]:
def convert_dates_to_days(bdate, x):
    time = x.time.values - bdate
    time = time.astype('timedelta64[s]').astype(float)/86400
    return x.assign_coords(time=time)


iop, nn, scam = [convert_dates_to_days(bdate, f) for f in [iop, nn, scam]]

Now, we need to broadcast all the precip fields

In [ ]:
ds = xr.Dataset({
    'nn': nn.prec,
    'cam': scam.prec,
    'iop': iop.PREC
}).dropna('time')\
.squeeze()

In [ ]:
ds.to_array().plot(col='variable', figsize=(3*3, 3*1.61), cmap='inferno_r', vmin=0)

It looks like the models all do a pretty good job of capturing the wave-like structure of the precipitation field. In fact the Neural network model seems to be worse the SCAM at capturing the precipitation peaks. This is clearer when we plot the time series for a single spatial location.

In [ ]:
axs = ds.isel(x=0).to_dataframe().drop(['x' ,'y'], 1)\
.plot(markevery=10, color='k', subplots=True)

for ax in axs:
    ax.set_yticks(np.arange(0, 200, 50));

As we can see, CAM overstimates many of the peaks in precipitation. This is probably because CAM listens to the erroneous forcing data more than the Neural network scheme, which has learned to overcome the errors in the forcings.